In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.functional import normalize

In [4]:
def read_data(file):
    return pd.read_csv(file, delimiter=',')

In [26]:
def split_data(data):
    x = data[:, :5]
    y = data[:, 5]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=37)
    return x_train, x_test, y_train, y_test

def transform(data):
    data = torch.tensor(data)
    # data = normalize(data)
    return data

In [27]:
def preprocess(filename):
    df_raw = read_data(filename)
    print(df_raw.head())
    d_raw = df_raw.to_numpy() # data to binary?
    x, x_test, y, y_test = split_data(d_raw)

    x, x_test = transform(x), transform(x_test)
    
    return x, x_test, y, y_test

In [52]:
datafile = 'data/basic_strategy_agent.csv'
x, x_test, y, y_test = preprocess(datafile)

   Player sum  Dealer card  Usable ace  Double down allowed  Split allowed  \
0          14            9           0                    1              0   
1          20            9           0                    0              0   
2          20            8           1                    1              0   
3          14           10           0                    1              0   

   Action  
0       1  
1       0  
2       0  
3       1  


In [55]:
def combine_input_and_label(x, y, n):
    y_one_hot = torch.eye(n, dtype=int)[y]
    return torch.concat((x, y_one_hot), 1)

def generate_neg_labels(y, n):
    y_ = y.copy()
    for i in range(len(y)):
        negative_labels = [j for j in range(n) if j != y[i]]
        y_[i] = np.random.choice(negative_labels)

    return y_

In [206]:
x_pos = combine_input_and_label(x, y, 4)

y_neg = generate_neg_labels(y, 4)
x_neg = combine_input_and_label(x, y_neg, 4)

print(f"x_pos: {x_pos[0]}")
print(f"x_neg: {x_neg[0]}")

x_pos: tensor([20,  9,  0,  0,  0,  1,  0,  0,  0])
x_neg: tensor([20,  9,  0,  0,  0,  0,  1,  0,  0])
